In [2]:
import pandas as pd
from pypfopt import EfficientFrontier
from sklearn.covariance import ShrunkCovariance
import os

In [3]:
data = pd.read_csv("features.csv")
open_cols = [col for col in data.columns if "Open" in col]
coins = [col.split("_")[1] for col in open_cols]

df_train = pd.read_csv("features_train.csv")
df_test1 = pd.read_csv("features_val.csv")
df_test2 = pd.read_csv("features_test.csv")

In [4]:
coins

['XRP',
 'NEO',
 'Stellar',
 'Dogecoin',
 'Tether',
 'XEM',
 'TRX',
 'NXT',
 'Ethereum',
 'PPC',
 'BNB',
 'BTC',
 'Litecoin']

In [16]:
# for all files in linear_pred folder get the predictions
# and add them to the dataframe
predictions = pd.DataFrame()
for file in os.listdir("linear_preds"):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join("linear_preds", file))
        predictions = pd.concat([predictions, df["predicted_y"]], axis=1)
        predictions.rename(columns={"predicted_y": file.split("_")[0]}, inplace=True)

In [19]:
predictions.dropna(inplace=True)
predictions

,NEO,BTC,TRX,Dogecoin,XRP,BNB,NXT,Litecoin,Tether,Stellar,Ethereum,XEM,PPC
1,0.017635,0.011444,-0.002060,-0.055933,-0.016152,-0.008133,-0.004392,0.017153,-0.001073,-0.031145,0.024588,-0.010439,-0.035402
2,-0.002044,0.008764,-0.012491,-0.029173,-0.004527,-0.010081,-0.002466,0.012259,-0.001739,-0.043922,0.021689,-0.020758,-0.023791
3,0.006131,0.004361,-0.012994,-0.018792,0.008487,-0.012972,-0.009455,0.007915,-0.002476,-0.029754,0.014126,-0.021247,-0.019775
4,0.007056,0.002411,-0.013086,-0.009809,-0.005086,-0.016531,0.026653,0.004484,-0.001127,-0.022600,0.011790,0.006716,-0.007196
5,0.012804,0.001111,-0.015346,-0.002406,-0.023138,-0.019999,0.005890,-0.005030,0.000184,-0.026313,0.002404,-0.000740,-0.004697
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.001938,-0.005393,0.015871,0.091264,0.005186,-0.009480,-0.001244,0.002945,-0.001215,-0.003508,0.008227,0.005127,-0.033405
66,-0.004855,-0.012327,0.007507,0.081530,-0.012031,-0.013033,0.022723,-0.003946,0.000073,-0.014561,0.002056,-0.002569,-0.017605
67,-0.035696,-0.019295,0.005182,0.083121,-0.024395,-0.014754,0.013774,-0.014781,-0.001374,-0.022267,0.002405,-0.005478,-0.020809
68,-0.017180,-0.010488,0.004794,0.102438,-0.026179,-0.011837,-0.014246,-0.018465,-0.000729,-0.013353,0.013059,-0.007882,-0.036867


In [ ]:
cov = ShrunkCovariance().fit(X)
cov_mtx = cov.covariance_

In [ ]:
pred_ret

In [ ]:
ef = EfficientFrontier(predictions[0], cov_mtx)
aw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)